# Cleanup the book ranking

The list contains a bunch of books that are not really books but common phrases (e.g. 'The Book'). They must be removed.

In [25]:
import re
import unidecode
from pathlib import Path

import pandas as pd

BOOK_COUNT_DF = Path('./data/book_counts_raw.csv')
BOOK_COUNT_CLEANED_DF = Path('./data/book_counts_cleaned.csv')
HAND_PICKED_AMBIVALENT_TITLES = Path('./data/faux_book_titles')
NON_ALPHANUMERIC = re.compile(r'[\W_]+')

def normalize_text(title: str) -> str:
    """
    Removes special characters, makes the string lowercase and returns the new string.
    """
    text = unidecode.unidecode(title)
    text = text.lower()
    text = re.sub(NON_ALPHANUMERIC, '', text)
    text = text.strip()

    if title[:4].lower() == 'the ':     # Starts with 'The ', 'A ' or 'An '
        text = text[3:]
    elif title[:3].lower() == 'an ':
        text = text[2:]
    elif title[:2].lower() == 'a ':
        text = text[1:]

    return text

In [26]:
book_ranking = pd.read_csv(BOOK_COUNT_DF, index_col=0)
book_ranking['Title'] = book_ranking['Title'].astype(str).str.strip('"')
book_ranking

,Year,Title,Normalized Title,Post or Comment,Occurrences
0,2021,The Heir,heir,post,4084
1,2020,The Heir,heir,post,3237
2,2022,The Heir,heir,post,3153
3,2022,The Heir,heir,comment,3039
4,2018,The Heir,heir,post,2221
...,...,...,...,...,...
83440,2021,Song of Kali,songofkali,post,1
83441,2021,Stravaganza,stravaganza,post,1
83442,2021,The heartbreaker,heartbreaker,post,1
83443,2021,Life is what you make it,lifeiswhatyoumakeit,post,1


In [27]:
# Ambivalent book titles that should be removed:
faux_titles = {normalize_text(title) for title in HAND_PICKED_AMBIVALENT_TITLES.read_text().splitlines(keepends=False)}
print(f'Collected {len(faux_titles):,} titles that have an ambivalent meaning and cannot be distinguished from normal phrases or sentences. They are removed from the ranking.')

Collected 858 titles that have an ambivalent meaning and cannot be distinguished from normal phrases or sentences. They are removed from the ranking.


In [28]:
filtered_book_ranking = book_ranking[~book_ranking['Normalized Title'].isin(faux_titles)]
filtered_book_ranking = pd.DataFrame(filtered_book_ranking.sort_values('Occurrences', ascending=False, ignore_index=True))
filtered_book_ranking.index = filtered_book_ranking.index + 1

filtered_book_ranking

,Year,Title,Normalized Title,Post or Comment,Occurrences
1,2012,The Hunger Games,hungergames,post,326
2,2020,1984,1984,post,310
3,2021,Dune,dune,post,310
4,2021,1984,1984,post,297
5,2020,Dune,dune,post,286
...,...,...,...,...,...
83163,2012,A Gift from Earth,giftfromearth,post,1
83164,2012,Flight of the Horse,flightofthehorse,post,1
83165,2012,Achilles' choice,achilleschoice,post,1
83166,2012,Foreign Agent,foreignagent,post,1


In [29]:
# Absolute ranking:
absolute_rank = pd.DataFrame(filtered_book_ranking.groupby(['Title', 'Normalized Title']).sum()).sort_values('Occurrences', ascending=False)
absolute_rank[['Occurrences']].reset_index()

C:\Users\timwi\AppData\Local\Temp\ipykernel_3604\103150657.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  absolute_rank = pd.DataFrame(filtered_book_ranking.groupby(['Title', 'Normalized Title']).sum()).sort_values('Occurrences', ascending=False)


,Title,Normalized Title,Occurrences
0,Dune,dune,2303
1,1984,1984,2228
2,The Hunger Games,hungergames,1173
3,Infinite jest,infinitejest,1045
4,The Hobbit,hobbit,992
...,...,...,...
21330,Silent Kill,silentkill,1
21331,Silence fallen,silencefallen,1
21332,Signs of love,signsoflove,1
21333,Signore Degli Anelli,signoredeglianelli,1


In [30]:
filtered_book_ranking.to_csv(BOOK_COUNT_CLEANED_DF)